In [ ]:
import numpy as np
import pandas as pd
import shutil
import matplotlib.pyplot as plt

In [ ]:
# Whether to plot matplotlib figures inside the notebook or create an output file for latex
USE_LATEX_ENGINE = True

if USE_LATEX_ENGINE:
    import matplotlib
    # matplotlib.use("pgf")
    matplotlib.rcParams.update({
        # "pgf.texsystem": "pdflatex",
        # 'pgf.rcfonts': False,
        'font.family': 'serif',
        'text.usetex': True,
    })

In [ ]:
normal_meta_data = '/mnt/c/Users/Jan/Daten/Geschäftlich/Capgemini/scripts/tmp/filtered_PADCHEST_chest_x_ray_images_labels_160K_01.02.19.csv'
pneumonia_meta_data = '/mnt/c/Users/Jan/Daten/Geschäftlich/Capgemini/scripts/tmp/filtered_pneumonia.csv'

# read csv files, engine='python' improves parsing, column differentiation, etc.
normal_meta_data = pd.read_csv(normal_meta_data, engine='python')
pneumonia_meta_data = pd.read_csv(pneumonia_meta_data, engine='python')

In [ ]:
def _generate_output_string(element, data):
    """
    Returns a string that contains information about the percentual share of a value in a list
    """
    if not isinstance(element, str): # make sure element is of type string, convert otherwise
        element = str(element)
    
    n_total = len(data)
    bool_list = (data == element)
    n_elements = sum(bool_list)

    return "{:<30}".format(str(n_elements) + '/' + str(n_total) + ' (' + "{:.2f}".format(((n_elements / n_total) * 100)) + '%)')

In [ ]:
def calculate_percentages(normal_data, pneumonia_data, additional_df=None):
    """
    checks if two lists contain the same set of values - returns a warning if the sets dont match - prints a string with information about the distribution if they match
    """
    if len(set(normal_data)) is not len(set(pneumonia_data)):
        print('WARNING! Categories of healthy and pneumonia data are not identical!')
        print('Normal: ' + str(set(normal_data)))
        print('Pneumonia: ' + str(set(pneumonia_data)))
        print('Number of appearences (normal):')
        n_normal_set = ''
        for category in set(normal_data):
            n_normal_set += "'" + str(category) + "': " + str(sum(normal_data == category)) + '\n'   # single quote: non-escaped string, double quote: escaped string
        print(n_normal_set)
        return
    else:
        print(set(normal_data))
    if additional_df is None:
        print('{:<21}'.format('') + '{:<30}'.format('normal') + '{:<30}'.format('pneumonia'))
    else:
        print('{:<21}'.format('') + '{:<30}'.format('normal (total)') + '{:<30}'.format('normal (filtered)') + '{:<30}'.format('pneumonia'))

    for element in set(normal_data):
        category_string = "{:<21}".format(element + ': ')
        if additional_df is None:
            print(category_string + _generate_output_string(element, normal_data) + _generate_output_string(element, pneumonia_data))
        else:
            print(category_string + _generate_output_string(element, additional_df) + _generate_output_string(element, normal_data) + _generate_output_string(element, pneumonia_data))

In [ ]:
def find_appearences(df, column, value, replace_nan_with=None):
    """
    searches for appearences of a specified value within the column of a dataframe - prints the number of appearences and the corresponding indexes
    """
    if replace_nan_with is not None:
        print('Replacing ' + str(df.isnull().sum().sum()) + ' appearances of NaN with: ' + str(replace_nan_with))
        df.fillna(replace_nan_with, inplace=True)   # be careful, this modifies the original dataframe that has been passed as variable
    value = str(value)
    print('found ' + str(sum(df[column] == value)) + ' rows')
    index_list = df.index[df[column] == value].tolist()
    print('Indexes: ' + str(index_list))

In [ ]:
def drop_from_column(df, column, value):
    """
    searches for appearences of a specified value within a column of a dataframe - drops each row where the value appeared from the dataframe
    """
    value = str(value)
    print('Dropping ' + str(sum(df[column] == value)) + ' rows')
    index_list = df.index[df[column] == value].tolist()
    df.drop(index_list, inplace=True)

In [ ]:
print('==== Distribution of projections ====\n')

calculate_percentages(normal_meta_data['Projection'], pneumonia_meta_data['Projection'])

find_appearences(normal_meta_data, 'Projection', 'UNK')
drop_from_column(normal_meta_data, 'Projection', 'UNK')

calculate_percentages(normal_meta_data['Projection'], pneumonia_meta_data['Projection'])

In [ ]:
print('==== Distribution of MethodLabel ====\n')

calculate_percentages(normal_meta_data['MethodLabel'], pneumonia_meta_data['MethodLabel'])

In [ ]:
print('==== Distribution of Patient Sex ====\n')

calculate_percentages(normal_meta_data['PatientSex_DICOM'], pneumonia_meta_data['PatientSex_DICOM'])

find_appearences(normal_meta_data, 'PatientSex_DICOM', '', 'U')     # 'U' for unknown

In [ ]:
calculate_percentages(normal_meta_data['PatientSex_DICOM'], pneumonia_meta_data['PatientSex_DICOM'])

find_appearences(normal_meta_data, 'PatientSex_DICOM', '')      # 'U' for unknown

drop_from_column(normal_meta_data, 'PatientSex_DICOM', 'U')     # drop 'U' and 'O' since they are heavily underrepresented
drop_from_column(normal_meta_data, 'PatientSex_DICOM', 'O')

calculate_percentages(normal_meta_data['PatientSex_DICOM'], pneumonia_meta_data['PatientSex_DICOM'])

In [ ]:
print('==== Distribution of ExposureTime ====\n')

calculate_percentages(normal_meta_data['ExposureTime'], pneumonia_meta_data['ExposureTime'])


In [ ]:
calculate_percentages(normal_meta_data['Exposure_DICOM'], pneumonia_meta_data['Exposure_DICOM'])

filtered_exposure_normal = [int(x) for x in normal_meta_data['Exposure_DICOM'] if str(x) != 'None']
filtered_exposure_pneumonia = [int(x) for x in pneumonia_meta_data['Exposure_DICOM'] if str(x) != 'None']
print(len(filtered_exposure_normal))
print(len(filtered_exposure_pneumonia))


In [ ]:
filtered_exposure_normal = sorted(filtered_exposure_normal)
filtered_exposure_pneumonia = sorted(filtered_exposure_pneumonia)

fig, ax_exp_time = plt.subplots()  # a figure with a single Axes

ax_exp_time.set_yscale('log')   # logarithmic scale on y axis

# bins must be -0.5 to center the x ticks, for detailled explanation see https://stackoverflow.com/questions/27083051/matplotlib-xticks-not-lining-up-with-histogram
ax_exp_time.hist(filtered_exposure_normal, np.arange(42)-0.5, alpha=0.5, label="Normal")
ax_exp_time.hist(filtered_exposure_pneumonia, np.arange(42)-0.5, alpha=0.5, color='r', label="Pneumonie")
ax_exp_time.set_xlabel('Strom * Zeit in mAs')
ax_exp_time.set_ylabel('Anzahl Röntgenbilder')
ax_exp_time.legend()
ax_exp_time.grid(True)

if USE_LATEX_ENGINE:
    plt.savefig("/mnt/c/Users/Jan/Daten/Dropbox/Master/3_Semester/Masterarbeit/Latex/python_output/padchest_combined_exposure.pdf")

In [ ]:
exposure_percentage_normal = [(filtered_exposure_normal.count(x)*100) / len(filtered_exposure_normal) for x in np.arange(42)]
exposure_percentage_pneumonia = [(filtered_exposure_pneumonia.count(x)*100) / len(filtered_exposure_pneumonia) for x in np.arange(42)]

fig, ax = plt.subplots()  # a figure with a single Axes

# bins must be -0.5 to center the x ticks, for detailled explanation see https://stackoverflow.com/questions/27083051/matplotlib-xticks-not-lining-up-with-histogram
ax.bar(np.arange(6)-0.125, exposure_percentage_normal[:6], width=0.25, label="Normal")
ax.bar(np.arange(6)+0.125, exposure_percentage_pneumonia[:6], color='r', width=0.25, label="Pneumonie")
ax.set_xlabel('Strom * Zeit in mAs')
ax.set_ylabel('Prozentualer Anteil')
ax.legend()
ax.grid(True)

if USE_LATEX_ENGINE:
    plt.savefig("/mnt/c/Users/Jan/Daten/Dropbox/Master/3_Semester/Masterarbeit/Latex/python_output/padchest_combined_exposure_percentage.pdf")

In [ ]:
print('==== Distribution of XRayTubeCurrent_DICOM ====\n')

calculate_percentages(normal_meta_data['XRayTubeCurrent_DICOM'], pneumonia_meta_data['XRayTubeCurrent_DICOM'])

In [ ]:
print('==== Distribution of Exposure_DICOM ====\n')

calculate_percentages(normal_meta_data['Exposure_DICOM'], pneumonia_meta_data['Exposure_DICOM'])


In [ ]:
print('==== Distribution of Modality_DICOM ====\n')

calculate_percentages(normal_meta_data['Modality_DICOM'], pneumonia_meta_data['Modality_DICOM'])

In [ ]:
print('==== Distribution of Manufacturer_DICOM ====\n')

calculate_percentages(normal_meta_data['Manufacturer_DICOM'], pneumonia_meta_data['Manufacturer_DICOM'])

In [ ]:
print('==== Distribution of PixelAspectRatio_DICOM ====\n')
calculate_percentages(normal_meta_data['PixelAspectRatio_DICOM'], pneumonia_meta_data['PixelAspectRatio_DICOM'])

In [ ]:
print('==== Distribution of Age ====\n')
calculate_percentages(normal_meta_data['PatientBirth'], pneumonia_meta_data['PatientBirth'])

In [ ]:
drop_from_column(normal_meta_data, 'PatientBirth', 'U') # drop unknown birth years, since there's only one entry

In [ ]:
normal_age_list = list()
number_of_normal = list()
for y in set(normal_meta_data['PatientBirth']):
    normal_age_list += [int(y)]
    number_of_normal += [sum(normal_meta_data['PatientBirth'] == y)]

print(normal_age_list)
print(number_of_normal)

In [ ]:
pneumonia_age_list = list()
number_of_pneumonia = list()
for y in set(pneumonia_meta_data['PatientBirth']):
    pneumonia_age_list += [int(y)]
    number_of_pneumonia += [sum(pneumonia_meta_data['PatientBirth'] == y)]

print(pneumonia_age_list)
print(number_of_pneumonia)

In [ ]:
plt.axis([1900, 2021 , 0, 1001])
plt.plot(normal_age_list, number_of_normal, label="Normal")
plt.plot(pneumonia_age_list, number_of_pneumonia, 'r', label="Pneumonie")
plt.ylabel('Anzahl Patienten')
plt.xlabel('Geburtsjahr')
plt.legend()
plt.grid(True)
# plt.show()

if USE_LATEX_ENGINE:
    plt.savefig('/mnt/c/Users/Jan/Daten/Dropbox/Master/3_Semester/Masterarbeit/Latex/python_output/year_of_birth_diagram.pdf')


In [ ]:
normal_ages = []
for study, birth in zip(normal_meta_data['StudyDate_DICOM'], normal_meta_data['PatientBirth']):
    normal_ages.append(int(str(study)[:4]) - int(birth))
print(normal_ages[:20])

In [ ]:
pneumonia_ages = []
for study, birth in zip(pneumonia_meta_data['StudyDate_DICOM'], pneumonia_meta_data['PatientBirth']):
    pneumonia_ages.append(int(str(study)[:4]) - int(birth))
print(pneumonia_ages[:20])

In [ ]:
x_axis_age = np.arange(max(set(normal_ages))+2)-0.5 # would be +1 for right aligned, but we want to center the x ticks -> +0.5
plt.hist(normal_ages, x_axis_age, alpha=0.5, label="Normal")
plt.hist(pneumonia_ages, x_axis_age, alpha=0.5, color='r', label="Pneumonie")
plt.ylabel('Anzahl Patienten')
plt.xlabel('Alter')
plt.legend()
plt.grid(True)
# plt.show()

if USE_LATEX_ENGINE:
    plt.savefig('/mnt/c/Users/Jan/Daten/Dropbox/Master/3_Semester/Masterarbeit/Latex/python_output/padchest_age_histogram.pdf')


In [ ]:
set(normal_meta_data)

In [ ]:
duplicate_patients = set(normal_meta_data['PatientID']) & set(pneumonia_meta_data['PatientID'])
print(str(len(duplicate_patients)) + ' Patients are both in Normal and Pneumonia data present')
print('Total number of Patients:\nNormal: ' + str(len(set(normal_meta_data['PatientID']))) + '\nPneumonia: ' + str(len(set(pneumonia_meta_data['PatientID']))))

In [ ]:
all_patient_ids = normal_meta_data['PatientID'].append(pneumonia_meta_data['PatientID'])
print(str(len(all_patient_ids)))
print(str(len(normal_meta_data['ImageID'])))
print(str(len(pneumonia_meta_data['ImageID'])))

# FROM HERE DRY RUN FILTERING TAKES PLACE

## Patients who have images with and without pneumonia
All samples of these patients labeled with normal will be removed from the dataset, to make sure their "normal" images don't show a light pneumonia.

In [ ]:
print("Dropping all samples labeled as normal whose patients also have samples with pneumonia label")
for id in duplicate_patients:
    drop_from_column(normal_meta_data, 'PatientID', id)

print("Dropped a total of " + str(len(duplicate_patients)) + " patients (" + str(len(all_patient_ids) - len(pneumonia_meta_data['PatientID']) - len(normal_meta_data['PatientID'])) + ' samples)')


## Experimental filtering and re-analysis

In [ ]:

filtered_normal_meta = []
copy_normal_meta_imagingdynamics = normal_meta_data.loc[normal_meta_data['Manufacturer_DICOM'] == "ImagingDynamicsCompanyLtd"]
copy_normal_meta_philips = normal_meta_data.loc[normal_meta_data['Manufacturer_DICOM'] == "PhilipsMedicalSystems"]

random_imagingdynamics = copy_normal_meta_imagingdynamics.sample(n = 1116)
random_philips = copy_normal_meta_philips.sample(n = 4125)
random_normal = random_imagingdynamics.append(random_philips)
print(len(random_imagingdynamics))
print(len(random_philips))
print(len(random_normal))


In [ ]:
print('==== Distribution of projections ====\n')
calculate_percentages(random_normal['Projection'], pneumonia_meta_data['Projection'])

In [ ]:
find_appearences(normal_meta_data, 'Projection', 'COSTAL')

In [ ]:
print(normal_meta_data.loc[8719]['ImageDir'])
print(normal_meta_data.loc[8719]['ImageID'])

In [ ]:
print(normal_meta_data.loc[35052]['ImageDir'])
print(normal_meta_data.loc[35052]['ImageID'])

In [ ]:
print(normal_meta_data.loc[21109]['ImageDir'])
print(normal_meta_data.loc[21109]['ImageID'])

In [ ]:
print(normal_meta_data.loc[446]['ImageDir'])
print(normal_meta_data.loc[446]['ImageID'])

In [ ]:
print(normal_meta_data.loc[27462]['ImageDir'])
print(normal_meta_data.loc[27462]['ImageID'])

In [ ]:
drop_from_column(normal_meta_data, 'Projection', 'COSTAL')


In [ ]:
find_appearences(pneumonia_meta_data, 'Projection', 'COSTAL')

In [ ]:
drop_from_column(pneumonia_meta_data, 'Projection', 'COSTAL')

In [ ]:
random_test = copy_normal_meta_imagingdynamics.sample(n=10)
print(random_test['PatientID'])

In [ ]:
random_test = copy_normal_meta_imagingdynamics.sample(n=10)
print(random_test['PatientID'])

In [ ]:

filtered_normal_meta = []
copy_normal_meta_imagingdynamics = normal_meta_data.loc[normal_meta_data['Manufacturer_DICOM'] == "ImagingDynamicsCompanyLtd"]
copy_normal_meta_philips = normal_meta_data.loc[normal_meta_data['Manufacturer_DICOM'] == "PhilipsMedicalSystems"]

random_imagingdynamics = copy_normal_meta_imagingdynamics.sample(n = 1116)
random_philips = copy_normal_meta_philips.sample(n = 4107)
random_normal = random_imagingdynamics.append(random_philips)

print("ImagingDynamics: " + str(len(random_imagingdynamics)) + '/' + str(len(copy_normal_meta_imagingdynamics)))
print("Philips; " + str(len(random_philips)) + '/' + str(len(copy_normal_meta_philips)))
print(len(random_normal))

# random_normal = normal_meta_data.sample(n = len(pneumonia_meta_data))

print('==== Distribution of projections ====\n')
calculate_percentages(random_normal['Projection'], pneumonia_meta_data['Projection'], normal_meta_data['Projection'])

In [ ]:
print('==== Distribution of Manufacturers ====\n')
calculate_percentages(random_normal['Manufacturer_DICOM'], pneumonia_meta_data['Manufacturer_DICOM'], normal_meta_data['Manufacturer_DICOM'])

In [ ]:
print('==== Distribution of MethodLabel ====\n')
calculate_percentages(random_normal['MethodLabel'], pneumonia_meta_data['MethodLabel'])

In [ ]:
print('==== Distribution of Patient Sex ====\n')
calculate_percentages(random_normal['PatientSex_DICOM'], pneumonia_meta_data['PatientSex_DICOM'])

In [ ]:
print('==== Distribution of Modality_DICOM ====\n')
calculate_percentages(random_normal['Modality_DICOM'], pneumonia_meta_data['Modality_DICOM'])

In [ ]:
print('==== Distribution of Manufacturer_DICOM ====\n')
calculate_percentages(random_normal['Manufacturer_DICOM'], pneumonia_meta_data['Manufacturer_DICOM'])

In [ ]:
print('==== Distribution of MethodLabel ====\n')
calculate_percentages(random_normal['MethodLabel'], pneumonia_meta_data['MethodLabel'])

In [ ]:
print('==== Distribution of MethodProjection ====\n')
calculate_percentages(random_normal['MethodProjection'], pneumonia_meta_data['MethodProjection'])

# Stratified Sampling from Kaggle

In [ ]:
'''
This module contains functions that computes stratified sampling of pandas dataframes.
'''
# Required libraries
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# Functions

In [ ]:
def __smpl_size(population, size):
    '''
    A function to compute the sample size. If not informed, a sampling 
    size will be calculated using Cochran adjusted sampling formula:
        cochran_n = (Z**2 * p * q) /e**2

        where:
            - Z is the z-value. In this case we use 1.96 representing 95%
            - p is the estimated proportion of the population which has an
                attribute. In this case we use 0.5
            - q is 1-p
            - e is the margin of error

        This formula is adjusted as follows:
        adjusted_cochran = cochran_n / 1+((cochran_n -1)/N)

        where:
            - cochran_n = result of the previous formula
            - N is the population size
    Parameters
    ----------
        :population: population size
        :size: sample size (default = None)
    Returns
    -------
    Calculated sample size to be used in the functions:
        - stratified_sample
        - stratified_sample_report
    '''
    if size is None:
        cochran_n = round(((1.96)**2 * 0.5 * 0.5)/ 0.02**2)
        n = round(cochran_n/(1+((cochran_n -1) /population)))
    elif size >= 0 and size < 1:
        n = round(population * size)
    elif size < 0:
        raise ValueError('Parameter "size" must be an integer or a proportion between 0 and 0.99.')
    elif size >= 1:
        n = size
    return n

In [ ]:
def stratified_sample_report(df, strata, size=None):
    '''
    Generates a dataframe reporting the counts in each stratum and the counts
    for the final sampled dataframe.

    Parameters
    ----------
    :df: pandas dataframe from which data will be sampled.
    :strata: list containing columns that will be used in the stratified sampling.
    :size: sampling size. If not informed, a sampling size will be calculated
        using Cochran adjusted sampling formula:
        cochran_n = (Z**2 * p * q) /e**2

        where:
            - Z is the z-value. In this case we use 1.96 representing 95%
            - p is the estimated proportion of the population which has an
                attribute. In this case we use 0.5
            - q is 1-p
            - e is the margin of error

        This formula is adjusted as follows:
        adjusted_cochran = cochran_n / 1+((cochran_n -1)/N)

        where:
            - cochran_n = result of the previous formula
            - N is the population size

    Returns
    -------
    A dataframe reporting the counts in each stratum and the counts
    for the final sampled dataframe.
    '''
    population = len(df)
    size = __smpl_size(population, size)
    tmp = df[strata]
    tmp['size'] = 1
    tmp_grpd = tmp.groupby(strata).count().reset_index()
    tmp_grpd['samp_size'] = round(size/population * tmp_grpd['size']).astype(int)
    return tmp_grpd


In [ ]:
def stratified_sample(df, strata, size=None, seed=None, keep_index= True):
    '''
    It samples data from a pandas dataframe using strata. These functions use
    proportionate stratification:
    n1 = (N1/N) * n
    where:
        - n1 is the sample size of stratum 1
        - N1 is the population size of stratum 1
        - N is the total population size
        - n is the sampling size
    Parameters
    ----------
    :df: pandas dataframe from which data will be sampled.
    :strata: list containing columns that will be used in the stratified sampling.
    :size: sampling size. If not informed, a sampling size will be calculated
        using Cochran adjusted sampling formula:
        cochran_n = (Z**2 * p * q) /e**2
        where:
            - Z is the z-value. In this case we use 1.96 representing 95%
            - p is the estimated proportion of the population which has an
                attribute. In this case we use 0.5
            - q is 1-p
            - e is the margin of error
        This formula is adjusted as follows:
        adjusted_cochran = cochran_n / 1+((cochran_n -1)/N)
        where:
            - cochran_n = result of the previous formula
            - N is the population size
    :seed: sampling seed
    :keep_index: if True, it keeps a column with the original population index indicator
    
    Returns
    -------
    A sampled pandas dataframe based in a set of strata.
    Examples
    --------
    >> df.head()
    	id  sex age city 
    0	123 M   20  XYZ
    1	456 M   25  XYZ
    2	789 M   21  YZX
    3	987 F   40  ZXY
    4	654 M   45  ZXY
    ...
    # This returns a sample stratified by sex and city containing 30% of the size of
    # the original data
    >> stratified = stratified_sample(df=df, strata=['sex', 'city'], size=0.3)
    Requirements
    ------------
    - pandas
    - numpy
    '''
    population = len(df)
    size = __smpl_size(population, size)
    tmp = df[strata]
    tmp['size'] = 1
    tmp_grpd = tmp.groupby(strata).count().reset_index()
    tmp_grpd['samp_size'] = round(size/population * tmp_grpd['size']).astype(int)

    # controlling variable to create the dataframe or append to it
    first = True 
    for i in range(len(tmp_grpd)):
        # query generator for each iteration
        qry=''
        for s in range(len(strata)):
            stratum = strata[s]
            value = tmp_grpd.iloc[i][stratum]
            n = tmp_grpd.iloc[i]['samp_size']

            if type(value) == str:
                value = "'" + str(value) + "'"
            
            if s != len(strata)-1:
                qry = qry + stratum + ' == ' + str(value) +' & '
            else:
                qry = qry + stratum + ' == ' + str(value)
        
        # final dataframe
        if first:
            stratified_df = df.query(qry).sample(n=n, random_state=seed).reset_index(drop=(not keep_index))
            first = False
        else:
            tmp_df = df.query(qry).sample(n=n, random_state=seed).reset_index(drop=(not keep_index))
            stratified_df = stratified_df.append(tmp_df, ignore_index=True)
    
    return stratified_df


In [ ]:
stratified_sample_report(normal_meta_data, ['Manufacturer_DICOM'], size=2000)